In [1]:
import pandas as pd
import numpy as np


df = pd.read_excel("21-24各省份录取数据(含专业组代码).xlsx", header=[0, 1])
df.columns = ['{}_{}'.format(str(a), str(b)) if str(a) != 'nan' else str(b) for a, b in df.columns]

In [8]:
df

,2024年计划数据_id,2024年计划数据_生源地,2024年计划数据_批次,2024年计划数据_科类,2024年计划数据_选科限制,2024年计划数据_院校代码,2024年计划数据_专业组代码,2024年计划数据_院校名称,2024年计划数据_专业代码,2024年计划数据_专业名称,...,2021年录取数据_最低分,2021年录取数据_最低位次,院校基础信息_所在省,院校基础信息_城市,院校基础信息_本科/专科,院校基础信息_隶属单位,院校基础信息_院校标签,院校基础信息_类型,院校基础信息_公私性质,院校基础信息_院校水平
0,7956360,黑龙江,本科批,物理,化,1001015,15,北京大学,30,工科试验班类,...,680,60,北京,北京海淀区,本科,教育部,"综合,985,211,双一流,国家重点,保研,研究生院,C9,五院四系,省部共建,101计划...",综合,公办,世界一流大学


In [9]:
import re

def smart_rename(col):
    # 只保留“id”或“生源地”
    if re.search(r'id$', col) or '生源地' in col:
        if 'id' == col or '生源地' == col:
            return col
        # 取最后一个下划线后的内容
        return col.split('_')[-1]
    return col

# 应用重命名
new_columns = [smart_rename(col) for col in df.columns]
df.columns = new_columns
print(df.columns)

Index(['id', '生源地', '2024年计划数据_批次', '2024年计划数据_科类', '2024年计划数据_选科限制',
       '2024年计划数据_院校代码', '2024年计划数据_专业组代码', '2024年计划数据_院校名称', '2024年计划数据_专业代码',
       '2024年计划数据_专业名称', '2024年计划数据_专业备注', '2024年计划数据_学制', '2024年计划数据_学费',
       '2024年计划数据_计划数', '2024年计划数据_新增专业', '2024年录取数据_录取最低分',
       '2024年录取数据_录取最低位次', '2024年录取数据_专业组最低分', '2024年录取数据_专业组最低位次',
       '2023年录取数据_专业组最低分', '2023年录取数据_专业组最低位次', '2023年录取数据_计划数',
       '2023年录取数据_最低分', '2023年录取数据_最低位次', '2022年录取数据_计划数', '2022年录取数据_最低分',
       '2022年录取数据_最低位次', '2021年录取数据_计划数', '2021年录取数据_最低分', '2021年录取数据_最低位次',
       '院校基础信息_所在省', '院校基础信息_城市', '院校基础信息_本科/专科', '院校基础信息_隶属单位', '院校基础信息_院校标签',
       '院校基础信息_类型', '院校基础信息_公私性质', '院校基础信息_院校水平'],
      dtype='object')


In [20]:
# 统计“2024年计划数据_选科限制”各label数量
label_counts = df['2024年计划数据_选科限制'].value_counts(dropna=False)
print(label_counts.keys())

Index([    nan,    '不限',     '化',   '物,化',     '物',     '史',     '政',     '生',
           '地',   '化与生',   '化和生', '物,化,生',   '化与物',   '化且生',   '物,生',   '政,史',
         '政与地',   '政和地',   '物,地', '政,史,地',   '史,政',   '化,生',   '史,地', '史,政,地',
         '化与地', '物,化,政',   '政且地',   '化和地',   '地与政',   '生与地',   '物,政',   '生与政',
         '化,地',   '生,地', '物,化,技', '物,化,地',   '生和政',   '生和地', '史,地,政',   '政,地',
         '生且地',   '地,物',     '历', '物,生,政',   '生,史',   '政,历'],
      dtype='object', name='2024年计划数据_选科限制')


In [23]:
# 将“2024年计划数据_选科限制”列转换为bit表示
# 使用 & 位运算，可以获取可选科目
def map_subject_to_bit(subject_str):
    subject_map = {
        "物": 1,  # 物理
        "化": 2,  # 化学
        "生": 4,  # 生物
        "政": 8,  # 政治
        "史": 16, # 历史
        "地": 32, # 地理
        "技": 64, # 技术
        "不限": 0 # 不限
    }
    if pd.isna(subject_str) or subject_str == "NaN":
        return 0 # 或者根据需求返回特定的值，比如 -1 或者 None
    
    # 处理特殊组合，例如 "化与生", "化和生", "化且生"
    subject_str = subject_str.replace("与", ",").replace("和", ",").replace("且", ",").replace("历", "史")

    bits = 0
    subjects = subject_str.split(',')
    for sub in subjects:
        sub = sub.strip()
        if sub in subject_map:
            bits |= subject_map[sub]
    return bits

# 应用到DataFrame
df['选科限制_bit'] = df['2024年计划数据_选科限制'].apply(map_subject_to_bit)

# 打印包含"物", "化", "生"的行的选科限制和转换后的bit值
print("包含 物, 化, 生 的选科限制及其bit表示:")
print(df[df['2024年计划数据_选科限制'].astype(str).str.contains('物') & 
         df['2024年计划数据_选科限制'].astype(str).str.contains('化') & 
         df['2024年计划数据_选科限制'].astype(str).str.contains('生')]
[['2024年计划数据_选科限制', '选科限制_bit']].head())

# 打印包含"政", "史", "地"的行的选科限制和转换后的bit值
print("\n包含 政, 史, 地 的选科限制及其bit表示:")
print(df[df['2024年计划数据_选科限制'].astype(str).str.contains('政') & 
         df['2024年计划数据_选科限制'].astype(str).str.contains('史') & 
         df['2024年计划数据_选科限制'].astype(str).str.contains('地')]
[['2024年计划数据_选科限制', '选科限制_bit']].head())

# 打印"不限"的选科限制和转换后的bit值
print("\n不限 的选科限制及其bit表示:")
print(df[df['2024年计划数据_选科限制'] == '不限'][['2024年计划数据_选科限制', '选科限制_bit']].head())

# 打印 NaN (空值) 的选科限制和转换后的bit值
print("\nNaN (空值) 的选科限制及其bit表示:")
print(df[df['2024年计划数据_选科限制'].isna()][['2024年计划数据_选科限制', '选科限制_bit']].head())

# 显示转换后 '选科限制_bit' 的值计数，以验证转换
print("\n选科限制_bit 值计数:")
print(df['选科限制_bit'].value_counts())

包含 物, 化, 生 的选科限制及其bit表示:
      2024年计划数据_选科限制  选科限制_bit
11934          物,化,生         7
11935          物,化,生         7
11936          物,化,生         7
24073          物,化,生         7
25824          物,化,生         7

包含 政, 史, 地 的选科限制及其bit表示:
       2024年计划数据_选科限制  选科限制_bit
9019            史,地,政        56
98510           政,史,地        56
98511           政,史,地        56
273566          政,史,地        56
354017          史,政,地        56

不限 的选科限制及其bit表示:
      2024年计划数据_选科限制  选科限制_bit
11934          物,化,生         7
11935          物,化,生         7
11936          物,化,生         7
24073          物,化,生         7
25824          物,化,生         7

包含 政, 史, 地 的选科限制及其bit表示:
       2024年计划数据_选科限制  选科限制_bit
9019            史,地,政        56
98510           政,史,地        56
98511           政,史,地        56
273566          政,史,地        56
354017          史,政,地        56

不限 的选科限制及其bit表示:
   2024年计划数据_选科限制  选科限制_bit
16             不限         0
17             不限         0
18             不限         0
19             不限  

In [85]:
# 选取湖北省的数据
hube_data = df[df['生源地'] == '湖北']
# 打印湖北省的数据
print(hube_data.head())

          id 生源地 2024年计划数据_批次 2024年计划数据_科类 2024年计划数据_选科限制 2024年计划数据_院校代码  \
406  8064481  湖北          本科批           物理             不限         A00304   
407  8064482  湖北          本科批           物理             不限         A00304   
408  8064483  湖北          本科批           物理             不限         A00304   
409  8064484  湖北          本科批           物理             不限         A00304   
410  8064485  湖北          本科批           物理             不限         A00304   

    2024年计划数据_专业组代码 2024年计划数据_院校名称 2024年计划数据_专业代码 2024年计划数据_专业名称  ...  \
406              04           清华大学             08        社会科学试验班  ...   
407              04           清华大学             09         理科试验班类  ...   
408              04           清华大学             10         理科试验班类  ...   
409              04           清华大学             11            法学类  ...   
410              04           清华大学             12            建筑类  ...   

    2021年录取数据_最低位次 院校基础信息_所在省 院校基础信息_城市 院校基础信息_本科/专科 院校基础信息_隶属单位  \
406            NaN         北京     北京

In [86]:
# 院校基础信息_院校标签 为tags，需要处理
tags_set = set()
def process_tags(tags_str):
    if pd.isna(tags_str) or tags_str == "NaN":
        return []
    # 分割标签字符串，去除空格
    tags = [tag.strip() for tag in tags_str.split(',') if tag.strip()]
    tags_set.update(tags)
    return tags
# 先看看有哪些标签
tags = hube_data['院校基础信息_院校标签'].value_counts()
print("\n湖北省院校标签统计:")
print(tags.keys())


湖北省院校标签统计:
Index(['综合,民办高校,本科', '理工,省重点,保研,研究生院,公立大学,本科,省重点', '理工,民办高校,本科',
       '综合,省属,公立大学,本科', '理工,省属,公立大学,本科', '综合,省重点,保研,研究生院,公立大学,本科,省重点',
       '师范,省属,公立大学,本科', '理工,211,双一流,国家重点,保研,研究生院,省部共建,公立大学,本科',
       '综合,211,双一流,国家重点,保研,研究生院,省部共建,公立大学,本科', '财经,省属,公立大学,本科',
       ...
       '医药,985,211,双一流,国家重点,公立大学,本科', '艺术,省属,八大美院,研究生院,公立大学,本科',
       '艺术,双一流,国家重点,保研,省部共建,公立大学,本科', '综合,省部共建,公立大学,本科', '艺术,省属,八大美院,公立大学,本科',
       '体育,公立大学,本科', '体育,省属,保研,公立大学,本科', '政法,双一流,国家重点,保研,公立大学,本科',
       '艺术,省重点,公立大学,本科,省重点', '医药,双一流,国家重点,保研,研究生院,公立大学,本科'],
      dtype='object', name='院校基础信息_院校标签', length=249)


In [87]:
hube_data['院校基础信息_院校标签_list'] = hube_data['院校基础信息_院校标签'].apply(process_tags)
print("tags_set 中的标签数量:", len(tags_set), "标签内容:", tags_set)
print("\n处理后的院校标签列表:")
hube_data['院校基础信息_院校标签_list'].head()

tags_set 中的标签数量: 35 标签内容: {'两电一邮', '985', '101计划', '建筑新八校', '农林', '综合', '八大美院', '国防七子', '语言', '公立大学', '军事', '体育', '政法', '理工', '民族', '省部共建', '211', '民办高校', 'E9', '省重点', '财经', '部委院校', '艺术', '双一流', '师范', '研究生院', '国家重点', '省属', '建筑老八校', '五院四系', '本科', 'C9', '保研', '医药', '其它'}

处理后的院校标签列表:


/var/folders/q3/z8wzr94n7g186jst74dr468r0000gn/T/ipykernel_43825/2762831013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hube_data['院校基础信息_院校标签_list'] = hube_data['院校基础信息_院校标签'].apply(process_tags)


406    [综合, 985, 211, 双一流, 国家重点, 保研, 研究生院, C9, 建筑老八校,...
407    [综合, 985, 211, 双一流, 国家重点, 保研, 研究生院, C9, 建筑老八校,...
408    [综合, 985, 211, 双一流, 国家重点, 保研, 研究生院, C9, 建筑老八校,...
409    [综合, 985, 211, 双一流, 国家重点, 保研, 研究生院, C9, 建筑老八校,...
410    [综合, 985, 211, 双一流, 国家重点, 保研, 研究生院, C9, 建筑老八校,...
Name: 院校基础信息_院校标签_list, dtype: object

In [88]:
# 处理 院校基础信息_院校水平 tags
levels_set = set()
def process_level(level_str):
    if pd.isna(level_str) or level_str == "NaN":
        return []
    # 分割标签字符串，去除空格
    levels = [level.strip() for level in level_str.split(',') if level.strip()]
    levels_set.update(levels)
    return levels
# 先看看有哪些院校水平
levels = hube_data['院校基础信息_院校水平'].value_counts()
print("\n湖北省院校水平统计:")
print(levels.keys())


湖北省院校水平统计:
Index(['区域知名大学', '区域高水平大学', '世界知名大学,中国高水平大学', '中国知名大学,区域一流大学', '世界知名大学,中国一流大学',
       '区域知名大学,独立学院', '世界知名高水平大学,中国顶尖大学', '世界高水平大学,中国顶尖大学', '区域知名大学,民办大学',
       '中国顶尖大学,民办大学', '中国高水平大学,民办大学', '区域高水平大学,独立学院', '中国一流大学,民办大学',
       '中国一流大学,独立学院', '世界一流大学', '中国顶尖大学,独立学院', '中国高水平大学,独立学院', '区域高水平大学,民办大学',
       '中国顶尖大学', '中国高水平大学', '其它', '中国一流大学'],
      dtype='object', name='院校基础信息_院校水平')


In [89]:
# 应用处理函数
hube_data['院校基础信息_院校水平_list'] = hube_data['院校基础信息_院校水平'].apply(process_level)
print("levels_set 中的院校水平数量:", len(levels_set), "院校水平内容:", levels_set)
print("\n处理后的院校水平列表:")
print(hube_data['院校基础信息_院校水平_list'].head())

levels_set 中的院校水平数量: 14 院校水平内容: {'世界高水平大学', '世界一流大学', '区域一流大学', '民办大学', '中国顶尖大学', '区域知名大学', '中国知名大学', '中国高水平大学', '独立学院', '世界知名高水平大学', '区域高水平大学', '其它', '中国一流大学', '世界知名大学'}

处理后的院校水平列表:
406    [世界一流大学]
407    [世界一流大学]
408    [世界一流大学]
409    [世界一流大学]
410    [世界一流大学]
Name: 院校基础信息_院校水平_list, dtype: object


/var/folders/q3/z8wzr94n7g186jst74dr468r0000gn/T/ipykernel_43825/3330060509.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hube_data['院校基础信息_院校水平_list'] = hube_data['院校基础信息_院校水平'].apply(process_level)


In [90]:
# Display unique values in the '2024年计划数据_学费' column
print("Unique values in '2024年计划数据_学费':")
# print(hube_data['2024年计划数据_学费'].unique())

# Identify values that are not purely numeric
# We can try to convert to numeric and see what fails
# pd.to_numeric will turn non-convertible strings into NaN if errors='coerce'
non_numeric_fees = hube_data[pd.to_numeric(hube_data['2024年计划数据_学费'], errors='coerce').isna()]['2024年计划数据_学费'].unique()

print("\nUnique non-numeric values found in '2024年计划数据_学费':")
print(non_numeric_fees)

Unique values in '2024年计划数据_学费':

Unique non-numeric values found in '2024年计划数据_学费':
['待定' '免费' '4000元']


In [91]:
# 替换特殊值，无法转换的待定为空值，使用Int32
# ['待定' '免费' '4000元']
hube_data['2024年计划数据_学费'] = (
    hube_data['2024年计划数据_学费']
    .replace({'待定': pd.NA, '免费': '0', '元': ''}, regex=True)
)
hube_data['2024年计划数据_学费'] = (
    pd.to_numeric(hube_data['2024年计划数据_学费'], errors='coerce').astype('Int32')
)

/var/folders/q3/z8wzr94n7g186jst74dr468r0000gn/T/ipykernel_43825/4256863055.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hube_data['2024年计划数据_学费'] = (
/var/folders/q3/z8wzr94n7g186jst74dr468r0000gn/T/ipykernel_43825/4256863055.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hube_data['2024年计划数据_学费'] = (


In [93]:
# 找到所有包含指定关键词的列
keywords = ['_计划数', '_录取最低分', '_最低分', '_录取最低位次', '_最低位次', '_专业组最低分', '_专业组最低位次']
target_columns = [col for col in hube_data.columns if any(k in col for k in keywords)]
print("匹配的列:", target_columns)

匹配的列: ['2024年计划数据_计划数', '2024年录取数据_录取最低分', '2024年录取数据_录取最低位次', '2024年录取数据_专业组最低分', '2024年录取数据_专业组最低位次', '2023年录取数据_专业组最低分', '2023年录取数据_专业组最低位次', '2023年录取数据_计划数', '2023年录取数据_最低分', '2023年录取数据_最低位次', '2022年录取数据_计划数', '2022年录取数据_最低分', '2022年录取数据_最低位次', '2021年录取数据_计划数', '2021年录取数据_最低分', '2021年录取数据_最低位次']


In [94]:
# 将上面列中数据转换为 Int32 类型, 保留空值
for col in target_columns:
    hube_data[col] = pd.to_numeric(hube_data[col], errors='coerce').astype('Int32')

/var/folders/q3/z8wzr94n7g186jst74dr468r0000gn/T/ipykernel_43825/3325553097.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hube_data[col] = pd.to_numeric(hube_data[col], errors='coerce').astype('Int32')


In [97]:
# 处理 2024年计划数据_新增专业 为 bool 类型
def process_new_major(new_major_str):
    if pd.isna(new_major_str) or new_major_str == "NaN":
        return False
    return new_major_str.strip().lower() in ['新增', 'yes', 'true']

hube_data['2024年计划数据_新增专业_bool'] = hube_data['2024年计划数据_新增专业'].apply(process_new_major)

/var/folders/q3/z8wzr94n7g186jst74dr468r0000gn/T/ipykernel_43825/3977574914.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hube_data['2024年计划数据_新增专业_bool'] = hube_data['2024年计划数据_新增专业'].apply(process_new_major)


In [98]:
# 检查data的数据类型
print("\n数据类型检查:")
print(hube_data.dtypes)


数据类型检查:
id                      int64
生源地                    object
2024年计划数据_批次           object
2024年计划数据_科类           object
2024年计划数据_选科限制         object
2024年计划数据_院校代码         object
2024年计划数据_专业组代码        object
2024年计划数据_院校名称         object
2024年计划数据_专业代码         object
2024年计划数据_专业名称         object
2024年计划数据_专业备注         object
2024年计划数据_学制           object
2024年计划数据_学费            Int32
2024年计划数据_计划数           Int32
2024年计划数据_新增专业         object
2024年录取数据_录取最低分         Int32
2024年录取数据_录取最低位次        Int32
2024年录取数据_专业组最低分        Int32
2024年录取数据_专业组最低位次       Int32
2023年录取数据_专业组最低分        Int32
2023年录取数据_专业组最低位次       Int32
2023年录取数据_计划数           Int32
2023年录取数据_最低分           Int32
2023年录取数据_最低位次          Int32
2022年录取数据_计划数           Int32
2022年录取数据_最低分           Int32
2022年录取数据_最低位次          Int32
2021年录取数据_计划数           Int32
2021年录取数据_最低分           Int32
2021年录取数据_最低位次          Int32
院校基础信息_所在省             object
院校基础信息_城市              object
院校基础信息_本科/专科           object
院

In [ ]:
# Assuming your DataFrame is named 'df' and has been processed as per your previous request.
# df = ... # Your DataFrame loading and processing code here

# Define the columns to select
selected_columns = [
    'id', '生源地', '2024年计划数据_批次', '2024年计划数据_科类', '选科限制_bit',
    '2024年计划数据_院校代码', '2024年计划数据_专业组代码', '2024年计划数据_院校名称', '2024年计划数据_专业代码',
    '2024年计划数据_专业名称', '2024年计划数据_专业备注', '2024年计划数据_学制', '2024年计划数据_学费',
    '2024年计划数据_计划数', '2024年计划数据_新增专业_bool', '2024年录取数据_录取最低分',
    '2024年录取数据_录取最低位次', '2024年录取数据_专业组最低分', '2024年录取数据_专业组最低位次',
    '2023年录取数据_专业组最低分', '2023年录取数据_专业组最低位次', '2023年录取数据_计划数',
    '2023年录取数据_最低分', '2023年录取数据_最低位次', '2022年录取数据_计划数', '2022年录取数据_最低分',
    '2022年录取数据_最低位次', '2021年录取数据_计划数', '2021年录取数据_最低分', '2021年录取数据_最低位次',
    '院校基础信息_所在省', '院校基础信息_城市', '院校基础信息_本科/专科', '院校基础信息_隶属单位', 
    '院校基础信息_院校标签_list', '院校基础信息_公私性质', '院校基础信息_院校水平_list'
]

# Select the desired columns from the DataFrame
# Ensure your DataFrame is named 'final_df' or change the name in the line below
df_selected = hube_data[selected_columns].copy() # Use .copy() to avoid SettingWithCopyWarning

# Rename columns to be ClickHouse friendly (optional, but good practice)
# Example: replace Chinese characters and spaces with underscores
rename_map = {
    '生源地': 'source_location',
    '2024年计划数据_批次': 'plan_batch',
    '2024年计划数据_科类': 'plan_subject_type',
    '选科限制_bit': 'plan_subject_restriction_bit',
    '2024年计划数据_院校代码': 'plan_school_code',
    '2024年计划数据_专业组代码': 'plan_major_group_code',
    '2024年计划数据_院校名称': 'plan_school_name',
    '2024年计划数据_专业代码': 'plan_major_code',
    '2024年计划数据_专业名称': 'plan_major_name',
    '2024年计划数据_专业备注': 'plan_major_remark',
    '2024年计划数据_学制': 'plan_duration',
    '2024年计划数据_学费': 'plan_tuition_fee',
    '2024年计划数据_计划数': 'admission_2024_plan_count',
    '2024年计划数据_新增专业_bool': 'plan_new_major',
    '2024年录取数据_录取最低分': 'admission_2024_min_score',
    '2024年录取数据_录取最低位次': 'admission_2024_min_rank',
    '2024年录取数据_专业组最低分': 'admission_2024_major_group_min_score',
    '2024年录取数据_专业组最低位次': 'admission_2024_major_group_min_rank',
    '2023年录取数据_专业组最低分': 'admission_2023_major_group_min_score',
    '2023年录取数据_专业组最低位次': 'admission_2023_major_group_min_rank',
    '2023年录取数据_计划数': 'admission_2023_plan_count',
    '2023年录取数据_最低分': 'admission_2023_min_score',
    '2023年录取数据_最低位次': 'admission_2023_min_rank',
    '2022年录取数据_计划数': 'admission_2022_plan_count',
    '2022年录取数据_最低分': 'admission_2022_min_score',
    '2022年录取数据_最低位次': 'admission_2022_min_rank',
    '2021年录取数据_计划数': 'admission_2021_plan_count',
    '2021年录取数据_最低分': 'admission_2021_min_score',
    '2021年录取数据_最低位次': 'admission_2021_min_rank',
    '院校基础信息_所在省': 'school_info_province',
    '院校基础信息_城市': 'school_info_city',
    '院校基础信息_本科/专科': 'school_info_level',
    '院校基础信息_隶属单位': 'school_info_affiliation',
    '院校基础信息_院校标签_list': 'school_info_tags_list',
    '院校基础信息_公私性质': 'school_info_public_private',
    '院校基础信息_院校水平_list': 'school_info_level_list'
}
df_selected.rename(columns=rename_map, inplace=True)

# Define ClickHouse table name
table_name = 'gaokao_data'

# Define ClickHouse schema based on DataFrame dtypes and column names
# Adjust types as necessary, especially for Nullable fields or specific ClickHouse types
# For example, fields that can be empty should be Nullable(String) or Nullable(Int64) etc.
# Array types for _list columns.
# Define the ClickHouse schema for the given DataFrame
schema_parts = []

# Iterate through each column of the DataFrame
for col_name in df_selected.columns:
    dtype = df_selected[col_name].dtype
    ch_type = 'String'  # Default to String for non-specific cases
    
    # Handling integer columns (Int32 or Nullable(Int32) instead of Int64)
    if pd.api.types.is_integer_dtype(dtype):
        if df_selected[col_name].isnull().any():  # Nullable case
            ch_type = 'Nullable(Int32)'  # Changed to Int32 for smaller range
        else:
            ch_type = 'Int32'  # Changed to Int32 for smaller range
    
    # Handling float columns (Float64 or Nullable(Float64))
    elif pd.api.types.is_float_dtype(dtype):
        if df_selected[col_name].isnull().any():  # Nullable case
            ch_type = 'Nullable(Float64)'
        else:
            ch_type = 'Float64'
    
    # Handling boolean columns (UInt8 or Nullable(UInt8))
    elif pd.api.types.is_bool_dtype(dtype):
        if df_selected[col_name].isnull().any():  # Nullable case
            ch_type = 'Nullable(UInt8)'  # ClickHouse uses UInt8 for boolean
        else:
            ch_type = 'UInt8'
    
    # Handling list columns (Array(String) or Array(OtherType))
    elif col_name.endswith('_list'):
        ch_type = 'Array(String)'  # Assuming the list contains strings; adjust if needed
    
    # Handling boolean for _bool columns
    elif col_name.endswith('_bool'):
        if df_selected[col_name].isnull().any():  # Nullable case
            ch_type = 'Nullable(UInt8)'
        else:
            ch_type = 'UInt8'
    
    # Handling string columns (String or Nullable(String))
    elif pd.api.types.is_object_dtype(dtype):
        if df_selected[col_name].isnull().any():  # Nullable case
            ch_type = 'Nullable(String)'
        else:
            ch_type = 'String'
    
    # Append the schema part
    schema_parts.append(f"`{col_name}` {ch_type}")

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    {', '.join(schema_parts)}
) ENGINE = MergeTree()
ORDER BY (id)
"""
print("\nClickHouse Create Table Query:")
print(create_table_query)


ClickHouse Create Table Query:

CREATE TABLE gaokao_data (
    `id` Int32, `source_location` String, `plan_batch` String, `plan_subject_type` String, `plan_subject_restriction_bit` Int32, `plan_school_code` String, `plan_major_group_code` String, `plan_school_name` String, `plan_major_code` String, `plan_major_name` String, `plan_major_remark` Nullable(String), `plan_duration` Nullable(String), `plan_tuition_fee` Nullable(Int32), `admission_2024_plan_count` Int32, `plan_new_major` UInt8, `admission_2024_min_score` Nullable(Int32), `admission_2024_min_rank` Nullable(Int32), `admission_2024_major_group_min_score` Nullable(Int32), `admission_2024_major_group_min_rank` Nullable(Int32), `admission_2023_major_group_min_score` Nullable(Int32), `admission_2023_major_group_min_rank` Nullable(Int32), `admission_2023_plan_count` Nullable(Int32), `admission_2023_min_score` Nullable(Int32), `admission_2023_min_rank` Nullable(Int32), `admission_2022_plan_count` Nullable(Int32), `admission_2022_min_

In [138]:
import clickhouse_connect
# ClickHouse connection details - MODIFY AS NEEDED
client = clickhouse_connect.get_client(host='127.0.0.1', port=18123, username='default', password='vfdeuiclgb') # Add username and password if required

In [ ]:
# 查询是否存在数据库gaokao
try:
    client.command("SELECT 1 FROM system.databases WHERE name = 'gaokao'")
    print("Database 'gaokao' exists.")
except Exception as e:
    print(f"Database 'gaokao' does not exist: {e}")

Database 'gaokao' exists.


In [ ]:
try:
    client.command(f"CREATE DATABASE IF NOT EXISTS gaokao")
    print("Database 'gaokao' created or already exists.")
except Exception as e:
    print(f"Error creating database: {e}")
# 切换到 gaokao 数据库
client.command("USE gaokao")

Database 'gaokao' created or already exists.


In [143]:
# 删除表
drop_table_query = f"DROP TABLE IF EXISTS {table_name};"
client.command(drop_table_query)

In [129]:
row = df_selected.values[0].tolist()
for i, v in enumerate(row):
    print(f"Index {i}: Type={type(v)}, Value={v}")

Index 0: Type=<class 'int'>, Value=8064481
Index 1: Type=<class 'str'>, Value=湖北
Index 2: Type=<class 'str'>, Value=本科批
Index 3: Type=<class 'str'>, Value=物理
Index 4: Type=<class 'int'>, Value=0
Index 5: Type=<class 'str'>, Value=A00304
Index 6: Type=<class 'str'>, Value=04
Index 7: Type=<class 'str'>, Value=清华大学
Index 8: Type=<class 'str'>, Value=08
Index 9: Type=<class 'str'>, Value=社会科学试验班
Index 10: Type=<class 'str'>, Value=(至善书院；经济学、社会学)
Index 11: Type=<class 'float'>, Value=nan
Index 12: Type=<class 'int'>, Value=5000
Index 13: Type=<class 'int'>, Value=1
Index 14: Type=<class 'bool'>, Value=False
Index 15: Type=<class 'pandas._libs.missing.NAType'>, Value=<NA>
Index 16: Type=<class 'pandas._libs.missing.NAType'>, Value=<NA>
Index 17: Type=<class 'int'>, Value=692
Index 18: Type=<class 'int'>, Value=71
Index 19: Type=<class 'int'>, Value=693
Index 20: Type=<class 'int'>, Value=85
Index 21: Type=<class 'int'>, Value=1
Index 22: Type=<class 'pandas._libs.missing.NAType'>, Value=<NA

In [144]:
import ast

def process_data_for_insert(df: pd.DataFrame):
    # 处理数据，确保与 ClickHouse 表匹配
    data_to_insert = []
    for row in df.values.tolist():
        processed_row = []
        for i, value in enumerate(row):
            col_name = df.columns[i]
            if value is pd.NA or isinstance(value, float) and pd.isna(value):
                processed_row.append(None)
            elif col_name.endswith('_list'):
                # 如果是_list类型，确保值是列表
                if isinstance(value, str):
                    try:
                        processed_row.append(ast.literal_eval(value) if value else [])
                    except (ValueError, SyntaxError):
                        processed_row.append([])
                elif isinstance(value, list):
                    processed_row.append([str(item) for item in value])
                else:
                    processed_row.append([])
            else:
                processed_row.append(value)
        data_to_insert.append(processed_row)
    
    return data_to_insert


def insert_data_in_batches(client, table_name, df: pd.DataFrame, batch_size=1000):
    # 处理数据并进行插入
    processed_data = process_data_for_insert(df)
    print(f"准备插入 {len(processed_data)} 条数据到表 '{table_name}'.")
    # 按批次插入数据
    for start in range(0, len(processed_data), batch_size):
        end = min(start + batch_size, len(processed_data))
        batch = processed_data[start:end]
        print(batch[0])
        # 插入数据
        client.insert(table_name, batch, column_names=list(df.columns))
        print(f"Batch {start // batch_size + 1} 插入成功.")

    client.close()
    print("ClickHouse 操作已完成。")

# 创建表（如果不存在）
client.command(create_table_query)
print(f"Table '{table_name}' created or already exists.")
# 使用方法：
insert_data_in_batches(client, 'gaokao_data', df_selected, batch_size=1000)


Table 'gaokao_data' created or already exists.
准备插入 18430 条数据到表 'gaokao_data'.
[8064481, '湖北', '本科批', '物理', 0, 'A00304', '04', '清华大学', '08', '社会科学试验班', '(至善书院；经济学、社会学)', None, 5000, 1, False, None, None, 692, 71, 693, 85, 1, None, None, 1, 676, 101, 1, None, None, '北京', '北京海淀区', '本科', '教育部', ['综合', '985', '211', '双一流', '国家重点', '保研', '研究生院', 'C9', '建筑老八校', '省部共建', '101计划', '公立大学', '本科'], '公办', ['世界一流大学']]
Batch 1 插入成功.
[8068862, '湖北', '本科批', '物理', 6, 'F09508', '08', '天津医科大学', '13', '麻醉学', '(色盲、色弱考生不予录取；办学地点：气象台路校区)(学制：五年)', None, 5800, 1, False, None, None, 623, 8760, 635, 6817, 1, None, None, 1, 610, 5640, None, None, None, '天津', '天津和平区', '本科', '市政府', ['医药', '211', '双一流', '国家重点', '保研', '研究生院', '公立大学', '本科'], '公办', ['世界知名大学', '中国高水平大学']]
Batch 2 插入成功.
[8066360, '湖北', '本科批', '物理', 2, 'A19104', '04', '西北农林科技大学', '11', '数学与应用数学', None, None, 6600, 2, False, None, None, 605, 15356, 609, 15469, 2, 615, 13191, None, None, None, None, None, None, '陕西', '陕西咸阳市', '本科', '教育部', ['农林', '985', '211'